### Description:

AES-128 algorithm with a hardcoded key is available as service.

To perform encryption, the service requires 16-bytes plaintext (in hex):

```
    https://training.usec.ch/student/sca1?ptext=00112233445566778899aabbccddeeff
```    
The result is JSON data
```
    {"ctext":"EB944E460582EE9FF1C65AC368ADA13C","trace":"4FE6247265A174AE9D32A3B105477D8F"}

```    
where:
```
    EB944E460582EE9FF1C65AC368ADA13C is a ciphertext
    
    4FE6247265A174AE9D32A3B105477D8F is a trace (information left by a programmer)
```
### Tips:

A 'trace' is a side-channel information left by a programmer to debug the implementation.

In this example a trace is a 10th round input state of the AES-128 algorithm, i.e., a State before the Sbox operation in the last round.

### Task:

Your task is to find the Master key (round key 0) embedded into the binary. 

The master key is in the form of SCA{XXXXXXXXXX}, where X is an ASCII printable symbol.

### Leakage illustration

<img src="support/Slide.png">

In [3]:
import numpy as np
import binascii
import string

import sca_training
#----------------------------------------------------------------------------
# This function calls tested service either with a user-defined 
# plaintext (if plaintext satisfies all the requirements) or with a 
# predefined plaintext (the same as in the header above)
#
# INPUTS:
#     plaintext - a string of 32 symbols representing 16 hex bytes of ciphertext
#     verbose   - a flag to print values in the function call or not
# OUTPUTS:
#     output  - a raw JSON output
#     ctext   - resulted ciphertext converted to numpy array of uint8
#     trace   - a trace associated with encryption process
#----------------------------------------------------------------------------
def binary_aes128_encrypt(plaintext, verbose=False):
    import requests
    
    if all(c in string.hexdigits for c in plaintext) and len(plaintext) == 32:
        output = requests.get("https://training.usec.ch/student/sca1?ptext="+plaintext).json()
    else:
        output = requests.get("https://training.usec.ch/student/sca1?ptext=00112233445566778899aabbccddeeff").json()
    
    #Transform the result into numpy array
    ctext = np.frombuffer(binascii.unhexlify(output["ctext"]), dtype=np.uint8)
    trace = np.frombuffer(binascii.unhexlify(output["trace"]), dtype=np.uint8)

    if (verbose):
        print('Binary output:', output)
        print('Ciphertext as numpy array:', ctext)
        print('Trace as numpy array:', trace)
    
    return output, ctext, trace


#----------------------------------------------------------------------------
# This function calls supportive library to compute a master key from
# the last round key.
#
# INPUTS:
#     last_round_key - a numpy array of 16 elements representing 16 hex bytes of
#                      the last round key, i.e., Round Key 10
# OUTPUTS:
#     key_schedule  - 11 round keys of 16 bytes each (key_schedule[0,0,:] is the
#                     master key
#----------------------------------------------------------------------------
# key_schedule = sca_training.inverse_key_expansion(last_round_key)

In [4]:
output, ctext, trace = binary_aes128_encrypt("010203040506070809000a0b0c0d0e1f", verbose=True)

Binary output: {'ctext': 'D1AEF6A6BD1CBBAC960C87E554928438', 'trace': 'B6CB6579085051113440FB755B1188A8'}
Ciphertext as numpy array: [209 174 246 166 189  28 187 172 150  12 135 229  84 146 132  56]
Trace as numpy array: [182 203 101 121   8  80  81  17  52  64 251 117  91  17 136 168]


### ATTACK CODE
Here you need to implement your attack which gives you the last round key.

Once the last round key is found - you need to compute the master key: this can be done with the function get_master_key()

### Programming tips
#### Numpy code
* np.arange(256).astype(np.uint8)
* np.bitwise_xor(m1, m2)
* np.where()

#### Code prepared for training
* sca_training.invSbox[sbox_out]
* sca_training.shift_rows(trace)

#### Various cod
* ''.join('{:02x}'.format(c) for c in int_array)

In [6]:
sbox_state = np.array([sca_training.Sbox_tab[trace[i]] for i in range(len(trace))], dtype=np.uint8)
shift_rows_state = sca_training.shift_rows(sbox_state)
k10 = np.bitwise_xor(ctext, shift_rows_state)
master_key = sca_training.get_master_key(k10)
''.join('{:02x}'.format(c) for c in master_key)


'5343417b53696d706c65526f756e647d'